## 6-1. 코랩 노트북
- 모델을 파인튜닝하려면 하드웨어 가속기를 사용해야 계산 속도를 높일 수 있습니다.
- 코랩에서는 GPU와 TPU 두 종류의 가속기를 지원합니다.
- 코랩 화면의 연결 GPU 메뉴 탭에서 런타임 > 런타임 유형 변경을 클릭합니다.
- 이후 GPU 혹은 TPU 둘 중 하나를 선택합니다.

* 파인튜닝(fine-tuning)은 기계 학습 모델을 특정 작업이나 데이터에 더 잘 맞도록 조정하는 과정을 의미.
주로 미리 학습된(pre-trained) 모델을 가져와서 새로운 작업에 맞게 조정하는 방식으로 사용

## 6-2. 환경 설정하기
### 1. 의존성 패키지 설치
- TPU 이외에 의존성 있는 패키지를 설치합니다.
- 코랩 환경에서는 명령어 맨 앞에 느낌표(!)를 붙이면 파이썬이 아닌, 배쉬 명령을 수행할 수 있습니다.

In [ ]:
# 코드 6-1 의존성 패키지 설치
!pip install ratsnlp

- 전 모델의 체크포인트는 구글 드라이브에 저장해 두었으므로 코드2를 실행해 코랩 노트북과 자신 구글 드라이브를 연동합니다.

In [ ]:
# 코드 6-2 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

- 6-3 을 실행하면 각종 설정을 할 수 있습니다.

In [ ]:
# 코드 6-3 인퍼런스 설정
from ratsnlp.nlpbook.ner import NERDeployArguments
args = NERDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-ner",
    max_seq_length=64,
)

#### 아래 코드를 실행하면 각종 설정을 할 수 있습니다.
#### 각 인자(argument)의 역할과 내용은 다음과 같습니다.
- 모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이.

## 6-3. 토크나이저 및 모델 불러오기
### 1. 토크나이저 로드
- 실행하면 토크나이저를 초기화할 수 있습니다.

In [ ]:
# 코드 6-4 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

- 파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.
- 아래 코드를 실행하면 이전 장에서 파인튜닝한 모델의 체크포인트를 읽어들입니다.

In [ ]:
# 코드 6-5 체크포인트 로드
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)

- 코드 6-6를 수행하면 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델의 설정 값들을 읽어들일 수 있습니다.
- 이어 6-7번 코드를 실행하면 해당 설정값대로 BERT 모델을 초기화합니다.

In [ ]:
# 코드 6-6 BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

In [ ]:
# 코드 6-7 BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

- 코드 6-8을 수행하면 코드 6-6에서 초기화한 BERT 모델에 코드 6-5의 체크포인트(fine_tuned_model_ckpt)를 읽어들이게 됩니다.
- 이어 코드 6-9를 실행하면 모델이 평가 모드로 전환됩니다.
- 드롭아웃 등 학습 때만 사용하는 기법들을 무효화하는 역할을 합니다.

In [ ]:
# 코드 6-8 체크포인트 읽기
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

In [ ]:
# 코드 6-9 EVAL MODE
model.eval()

## 6-4. 모델 출력값 만들고 후처리하기
###1. 인퍼런스 과정
- 개체명 인식 모델의 출력은 각 토큰이 어떤 개체명 태그에 속하는지 확률입니다.
- 인퍼런스를 하려면 확률값의 각 요소값이 어떤 태그에 대응하는지 정보를 알고 있어야 합니다.
- 이와 관련해 코드10을 실행하면 정수(integer) 인덱스를 레이블에 매핑하는 사전(dictionary)을 만듭니다.



In [ ]:
# 코드 6-10 label 맵 작성
labels = [label.strip() for label in open(args.downstream_model_labelmap_fpath, "r").readlines()]
id_to_label = {}
for idx, label in enumerate(labels):
  if "PER" in label:
    label = "인명"
  elif "LOC" in label:
    label = "지명"
  elif "ORG" in label:
    label = "기관명"
  elif "DAT" in label:
    label = "날짜"
  elif "TIM" in label:
    label = "시간"
  elif "DUR" in label:
    label = "기간"
  elif "MNY" in label:
    label = "통화"
  elif "PNT" in label:
    label = "비율"
  elif "NOH" in label:
    label = "기타 수량표현"
  elif "POH" in label:
    label = "기타"
  else:
    label = label
  id_to_label[idx] = label

- 6-10 수행 결과(id_to_label)는 다음과 같습니다.

```
{0: ‘[CLS]’, 1: ‘[SEP]’, 2: ‘[PAD]’, 3: ‘[MASK]’, 4: ‘O’, 5: ‘인명’, 6: ‘기타 수량표현’, 7: ‘기타’, 8: ‘기관명’, 9: ‘날짜’, 10: ‘지명’, 11: ‘통화’, 12: ‘비율’, 13: ‘시간’, 14: ‘기간’, 15: ‘인명’, 16: ‘기타 수량표현’, 17: ‘기타’, 18: ‘기관명’, 19: ‘날짜’, 20: ‘지명’, 21: ‘통화’, 22: ‘비율’, 23: ‘시간’, 24: ‘기간’}
```

- 6-11은 인퍼런스 과정을 정의한 함수입니다.
- 문장(sentence)을 입력받아 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만듭니다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다.
- 모델 출력값(outputs.logits)은 토큰 각각에 대해 반환되며 소프트맥스 함수 적용 이전의 로짓(logit)입니다
- 여기에 소프트맥스 함수를 써서 해당 토큰이 특정 개체명 태그일 확률 형태로 바꿉니다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 특정 태그의 위치일 경우 해당 태그가 출력되도록 pred 값을 만듭니다.



In [ ]:
# 코드 6-11 INFERENCE

def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        probs = outputs.logits[0].softmax(dim=1)
        top_probs, preds = torch.topk(probs, dim=1, k=1)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        predicted_tags = [id_to_label[pred.item()] for pred in preds]
        result = []
        for token, predicted_tag, top_prob in zip(tokens, predicted_tags, top_probs):
            if token not in [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token]:
                token_result = {
                    "token": token,
                    "predicted_tag": predicted_tag,
                    "top_prob": str(round(top_prob[0].item(), 4)),
                }
                result.append(token_result)
    return {
        "sentence": sentence,
        "result": result,
    }

## 6-5. 웹 서비스 시작하기
- 위의 코드에서 정의한 인퍼런스 함수(inference_fn)을 가지고 코드 6-12을 실행하면 플라스크(flask)라는 파이썬 라이브러리의 도움을 받아 웹 서비스를 띄울 수 있습니다.

In [ ]:
# 코드 6-12 웹 서비스
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()